# Test in CUB Dataset

In [1]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import imp
import utils
import torch.optim as optim
import numpy as np
imp.reload(utils)
from sklearn.manifold import TSNE


INPUT_DIM = 640
OUTPUT_DIM = 10
base_features_path = "./data/cub.pkl"

In [ ]:
datater = utils.Data()
f_matrix,labels,means,covs = datater.read_data(base_features_path,INPUT_DIM,0,10,'cub')
f_matrix = (f_matrix-np.min(f_matrix))/np.max(f_matrix)
print(means.shape)
x_train,x_test,y_train,y_test = utils.split_dataset(f_matrix,labels,4) # 划分数据集
print(x_train.shape,x_test.shape)
_,x_some,_,y_some = utils.split_dataset(x_train,y_train,25)
print(x_some.shape)

In [ ]:
# Test of undistilled data 
acc = utils.logistic_test(x_train,y_train,x_test,y_test)
acc_some = utils.logistic_test(x_some,y_some,x_test,y_test)
print("Logistic Acc (all):{0},Logistic Acc (some):{1}".format(acc,acc_some))

In [ ]:
# Train the PE-based model
model = utils.MCC(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-3)
utils.train(model,optimizer,x_train,y_train,0.1)
W1 = model.fc1.weight.clone().detach().numpy()

In [ ]:
# visulization data fearures with t-SNE
f_new = f_matrix@W1.T
tsne = TSNE()
out = tsne.fit_transform(f_new)
plt.figure(figsize=(10,10))
for i in range(10):
    indices = labels == i
    x, y = out[indices].T
    plt.scatter(x, y, label=str(i))
plt.legend()
plt.show()

In [ ]:
# Test the PE-based method with sampling data
x_r_new = x_some@W1.T
x_t_new = x_test@W1.T
acc = utils.logistic_test(x_r_new,y_some,x_t_new,y_test)
print("The acc of PEM (logistic) is:",acc)

### New_class_add

In [ ]:
# Get a new class data
n_matrix,n_labels,_,_ = datater.read_data(base_features_path,INPUT_DIM,10,11,'mini')
nx_train,nx_test,ny_train,ny_test = utils.split_dataset(n_matrix,n_labels,4)
nnx_train = np.vstack((x_some,nx_train))
nny_train = np.hstack((y_some,ny_train))
nx_test = np.vstack((x_test,nx_test))
ny_test = np.hstack((y_test,ny_test))
acc = utils.logistic_test(nnx_train,nny_train,nx_test,ny_test) # 测试Logistic model的准确性
print("Logistic Acc (some + new_class) acc:{0}".format(acc))

In [ ]:
# perform the smapling process
upper = utils.Distill(10)
x_up,y_up = upper.upsampling(x_some@W1.T,y_some,300)
npx_train = np.vstack((x_up,nx_train@W1.T))
npy_train = np.hstack((y_up,ny_train))
acc = utils.logistic_test(npx_train,npy_train,nx_test@W1.T,ny_test) # 测试Logistic model的准确性
print("Logistic PEM Acc (sampling) acc:{0}".format(acc))

In [ ]:
# Train the model again
npx = npx_train
model = utils.MCC(input_dim=OUTPUT_DIM, output_dim=OUTPUT_DIM)
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
utils.train(model,optimizer,npx,npy_train)
W2 = model.fc1.weight.clone().detach().numpy()

In [ ]:
# Test the result
x_r_new = npx@W2.T
x_t_new = nx_test@W1.T
x_t_new = x_t_new@W2.T
acc = utils.logistic_test(x_r_new,npy_train,x_t_new,ny_test)
print("Logistic PEM Acc (smapling + PEM) acc:{0}".format(acc))

### Merge a class

In [ ]:
# To add merge class, we first split the data set.
n_matrix,n_labels,_,_ = datater.read_data(base_features_path,INPUT_DIM,17,18,'mini')
n_labels = 8*np.ones(n_labels.shape)
nx_train,nx_test,ny_train,ny_test = utils.split_dataset(n_matrix,n_labels,4)
nnx_train = np.vstack((x_some,nx_train))
nny_train = np.hstack((y_some,ny_train))
nx_test = np.vstack((x_test,nx_test))
ny_test = np.hstack((y_test,ny_test))
acc = utils.logistic_test(nnx_train,nny_train,nx_test,ny_test) # 测试Logistic model的准确性
print("Logistic Acc (some + new_class) acc:{0}".format(acc))

In [ ]:
# perform the smapling process
upper = utils.Distill(10)
x_up,y_up = upper.upsampling(x_some@W1.T,y_some,300)
npx_train = np.vstack((x_up,nx_train@W1.T))
npy_train = np.hstack((y_up,ny_train))
acc = utils.logistic_test(npx_train,npy_train,nx_test@W1.T,ny_test) # 测试Logistic model的准确性
print("Logistic PEM Acc (sampling) acc:{0}".format(acc))

In [ ]:
# Train the model again
npx = npx_train
model = utils.MCC(input_dim=OUTPUT_DIM, output_dim=OUTPUT_DIM,)
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
utils.train(model,optimizer,npx,npy_train)
W2 = model.fc1.weight.clone().detach().numpy()

In [ ]:
# Test the result
x_r_new = npx@W2.T
x_t_new = nx_test@W1.T
x_t_new = x_t_new@W2.T
acc = utils.logistic_test(x_r_new,npy_train,x_t_new,ny_test)
print("Logistic PEM Acc (smapling + PEM) acc:{0}".format(acc))

### DataSet Sampling

In [ ]:
# Here we show the index. The index is the relative in idx in a class.
# The upper.refine() will get the real features.
upper = utils.Distill(10)
x_id,y_id = upper.select_sample(x_train@W1.T,y_train)
print(x_id,y_id)